In [1]:
# import numpy as np
import pandas as pd
import requests
import datetime
import time

import env

In [2]:
baseUrl = 'https://www.googleapis.com/youtube/v3/search'
youtubeDataBaseParams = {
  'key': env.token,
  'part': 'snippet',
  'order': 'viewCount',
  'regionCode': 'KR',
  'type': 'video'
}

numOfVideoPerMovie = 5
searchKeywords = ['영화', '명장면', '리뷰', '해석', '의미']

In [3]:
movieInfo = pd.read_csv('movieInfo.csv').loc[:,['영화명', '개봉일']]
movieNames = list(movieInfo.loc[:, '영화명'])

In [5]:
# 중복 제거
# 한 키워드당 2개 -> 한 영화당 영상 최대 10개
# 한 영화당 요청 5개 -> 한 영화당 5*20 = 100초 -> 영화 10개당 17분

def req_youtube():
  columns = ['영화 제목', '검색 키워드', '영상 id', '영상 제목', '생성된 날짜']
  res_data = pd.DataFrame(columns=columns)
  tokenNum = 0

  for movieName in movieNames[20:60]:
    print(movieName, len(res_data))
    for keyword in searchKeywords:
      publishedAfter = list(movieInfo[movieInfo['영화명'] == movieName].loc[:, '개봉일'])[0]
      year, month, day = publishedAfter.split('-')
      publishedBefore = (datetime.datetime(int(year), int(month), int(day)) + datetime.timedelta(days=30)).strftime('%Y-%m-%d')
      q = ' '.join([movieName, keyword])

      youtubeDataBaseParams['q'] = q
      youtubeDataBaseParams['publishedAfter'] = publishedAfter+'T00:00:00Z'
      youtubeDataBaseParams['publishedBefore'] = publishedBefore+'T00:00:00Z'

      time.sleep(20)
      res = requests.get(baseUrl, youtubeDataBaseParams)
      data = res.json()

      while 'error' in data.keys():
        print('[HTTPError] token 교체')
        tokenNum += 1
        if tokenNum > 2: 
          print('[Error] token 오늘 할당량 끝났다~~', movieName, keyword)
          return res_data

        youtubeDataBaseParams['key'] = env.token[tokenNum]
        res = requests.get(baseUrl, youtubeDataBaseParams)
        data = res.json()

      numOfNewId = 0
      if 'items' in data.keys():
        for item in data['items']:
          res_data_video_ids = list(res_data.loc[res_data['영화 제목'] == movieName,'영상 id'])

          if item['id']['videoId'] not in res_data_video_ids:
            item_data = [[movieName, q, item['id']['videoId'], item['snippet']['title'], item['snippet']['publishTime']]]
            res_data = res_data.append(pd.DataFrame(item_data, columns=columns),ignore_index=True)
            numOfNewId += 1
            if numOfNewId >= 2: break

  return res_data

res_data = req_youtube()

타짜: 원 아이드 잭 0
신의 한 수: 귀수편 9
알리타: 배틀 엔젤 19
내안의 그놈 27
뺑반 32
변신 37
주먹왕 랄프 2: 인터넷 속으로 43
걸캅스 51
사자 57
아쿠아맨 65
드래곤 길들이기 3 73
나의 특별한 형제 81
어스 87
말레피센트 2 96
유열의 음악앨범 105
퍼펙트맨 115
포드 V 페라리 121
생일 128
힘을 내요, 미스터 리 135
항거:유관순 이야기 139
[HTTPError] token 교체
장사리 : 잊혀진 영웅들 147
마이펫의 이중생활 2 153
롱 리브 더 킹: 목포 영웅 160
쥬만지: 넥스트 레벨 170
존 윅 3: 파라벨룸 178
남산의 부장들 187
다만 악에서 구하소서 197
반도 207
히트맨 216
테넷 221
백두산 229
#살아있다 239
강철비2: 정상회담 248
담보 256
닥터 두리틀 260
삼진그룹 영어토익반 266
정직한 후보 275
도굴 280
클로젯 284
오케이 마담 290
[HTTPError] token 교체
[HTTPError] token 교체
[Error] token 오늘 할당량 끝났다~~ ['극한직업', '어벤져스: 엔드게임', '엑시트', '스파이더맨: 파 프롬 홈', '캡틴 마블', '조커', '봉오동 전투', '라이온 킹', '나쁜 녀석들: 더 무비', '82년생 김지영', '분노의 질주: 홉스&쇼', '토이 스토리 4', '돈', '악인전', '가장 보통의 연애', '말모이', '증인', '블랙머니', '터미네이터: 다크 페이트', '사바하', '타짜: 원 아이드 잭', '신의 한 수: 귀수편', '알리타: 배틀 엔젤', '내안의 그놈', '뺑반', '변신', '주먹왕 랄프 2: 인터넷 속으로', '걸캅스', '사자', '아쿠아맨', '드래곤 길들이기 3', '나의 특별한 형제', '어스', '말레피센트 2', '유열의 음악앨범', '퍼펙트맨', '포드 V 페라리', '생일', '힘을 내요, 미스터 리', '항거:유관순 이야기', '장사리 : 잊혀진 영웅들', '마이

In [17]:
res_data

,영화 제목,검색 키워드,영상 id,영상 제목,생성된 날짜
0,타짜: 원 아이드 잭,타짜: 원 아이드 잭 영화,LtHuvUiqAls,역대 최악의 타짜 영화. 올 추석 대표 졸작 - 타짜: 원 아이드 잭 리뷰,2019-09-14T11:00:05Z
1,타짜: 원 아이드 잭,타짜: 원 아이드 잭 영화,uvVuRziAPm8,타짜1을 기억하는 사람들에게 타짜3가 졸작인 이유,2019-09-20T08:47:15Z
2,타짜: 원 아이드 잭,타짜: 원 아이드 잭 명장면,8zb3n5Hu5pY,"타짜1,2 명장면 모음 + 타짜3 하이라이트 (자체편집)",2019-09-13T07:00:21Z
3,타짜: 원 아이드 잭,타짜: 원 아이드 잭 리뷰,mueFS0qccLo,타짜: 원 아이드 잭 가이드 리뷰,2019-09-13T09:00:07Z
4,타짜: 원 아이드 잭,타짜: 원 아이드 잭 리뷰,XrUTaZKfgoU,"최악의 도박영화, 타짜: 원 아이드 잭의 문제는 캐릭터다!",2019-09-17T06:38:09Z
...,...,...,...,...,...
285,클로젯,클로젯 영화,ITWWNHwyV7Q,클로젯 가이드 리뷰,2020-02-10T09:00:07Z
286,클로젯,클로젯 리뷰,IQ-w-xTvTJE,클로젯 공포영화인데 하나도 안 무서운 이유 (귀신에 숨겨진 의미 해석),2020-02-08T00:59:29Z
287,클로젯,클로젯 리뷰,1N9cQgPLcjE,하정우에게 실망한 이유 - 클로젯,2020-02-15T09:33:21Z
288,클로젯,클로젯 해석,RVQBLkjlYUY,클로젯 _ 🍺영화 리뷰,2020-02-06T12:31:52Z


In [18]:
res_data.to_csv('./movieVideoList_20_58.csv', encoding='utf-8-sig')